In [2]:
import cv2
import numpy as np

In [11]:
import cv2
import numpy as np
import torch
from ultralytics import YOLO

DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = YOLO("../f1-strategy/weights/f1_detection_fine_tuned_best.pt")
model.to(DEVICE)

CAR_LENGTH_METERS = 5.5  # Longitud real del coche
FRAME_WIDTH = 640

def calculate_gap(box1, box2):
    """Calcula la distancia entre centros usando el ancho de los coches para la escala"""
    # Centros de las cajas
    cx1, cy1 = (box1[0] + box1[2])/2, (box1[1] + box1[3])/2
    cx2, cy2 = (box2[0] + box2[2])/2, (box2[1] + box2[3])/2
    
    # Distancia en píxeles
    pixel_distance = np.hypot(cx2 - cx1, cy2 - cy1)
    
    # Escala basada en el ancho promedio de los coches detectados
    avg_width = ((box1[2] - box1[0]) + (box2[2] - box2[0])) / 2
    scale = CAR_LENGTH_METERS / avg_width if avg_width != 0 else 0
    
    return pixel_distance * scale

def process_video_with_yolo(video_path):
    cap = cv2.VideoCapture(video_path)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break
        
        frame_resized = cv2.resize(frame, (FRAME_WIDTH, FRAME_WIDTH))
        results = model.predict(frame_resized, conf=0.4, verbose=False)[0]
        
        if results.boxes and len(results.boxes) > 0:
            boxes = results.boxes.xyxy.cpu().numpy()
            
            # Ordenar por posición vertical (coches más abajo primero - más cercanos)
            boxes = sorted(boxes, key=lambda x: x[3], reverse=True)
            
            # Dibujar cajas y gaps
            for i, box in enumerate(boxes):
                x1, y1, x2, y2 = map(int, box)
                cv2.rectangle(frame_resized, (x1, y1), (x2, y2), (255, 0, 0), 2)
                
                # Solo si hay siguiente coche
                if i < len(boxes)-1:
                    next_box = boxes[i+1]
                    gap = calculate_gap(box, next_box)
                    
                    # Puntos de conexión
                    cx1, cy1 = int((x1+x2)/2), int((y1+y2)/2)
                    cx2, cy2 = int((next_box[0]+next_box[2])/2), int((next_box[1]+next_box[3])/2)
                    
                    # Línea diagonal entre centros
                    cv2.line(frame_resized, (cx1, cy1), (cx2, cy2), (0,255,0), 2)
                    
                    # Texto en el punto medio
                    mid_x, mid_y = (cx1+cx2)//2, (cy1+cy2)//2
                    cv2.putText(frame_resized, f"{gap:.1f}m", (mid_x, mid_y),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

        cv2.imshow("F1 Gap Detection", frame_resized)
        if cv2.waitKey(1) == ord('q'): break
            
    cap.release()
    cv2.destroyAllWindows()

# Ejecutar con tu video
#video_path = "../f1-strategy/data/videos/best_overtakes_2023.mp4.f399.mp4"
video_path = "../f1-strategy/data/videos/spain_2023_race.mp4.f399.mp4"

process_video_with_yolo(video_path)